In [ ]:
# Assignment 3 - 7313

In [1]:
# STEP 0 - Importing Libraries

import matplotlib.pyplot as plt  
import numpy as np 
import pandas as pd 
import seaborn as sns 
import sklearn.metrics as metrics
import xgboost as xgb
import matplotlib.pyplot as plt
import calendar
import json     

from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from gettext import install

In [79]:
# STEP 0 - Downloading data as CSV files from SQL and then importing into python. 

# Importing the CSV file. Column names will be automatically inferred from the header row.

# Apartment
df_apartment = pd.read_csv('Apartment.csv') 

# AnnualReport
df_annualreport = pd.read_csv('AnnualReport.csv') 

# HousingAssociation
df_housingassociation = pd.read_csv('HousingAssociation.csv') 

# Dropping features that will not be imputed
df_annualreport = df_annualreport.drop(['long_term_debt_other', 'long_term_real_estate_debt', 'number_of_rental_units', 'plot_is_leased', 'savings', 
                                        'total_commercial_area', 'total_loan', 'total_rental_area'], axis=1)

df_housingassociation = df_housingassociation.drop(['name', 'housing_coop_id'], axis=1) 

In [80]:
# STEP 1 - Merging the dataframes  

# Renaming from housing_association_org_number to org_number in df_apartment
df_apartment = df_apartment.rename(columns={'housing_association_org_number': 'org_number'}) 

# Reformating fiscal_year by pushing forward the one year, while preserves NaN
df_annualreport['fiscal_year_plus_one'] = df_annualreport['fiscal_year'] + 1 

# Convert sell_date column to datetime format 
df_apartment['sell_date'] = pd.to_datetime(df_apartment['sell_date']) 

# Extract the year from sell_date
df_apartment['sell_year'] = df_apartment['sell_date'].dt.year.astype('int64') 

# Sort fiscal_year_plus_one and org_number in ascending order in df_annualreport
df_annualreport = df_annualreport.sort_values(by=['fiscal_year_plus_one', 'org_number'])

# Sort sell_year and org_number in ascending order in df_apartment 
df_apartment = df_apartment.sort_values(by=['sell_year','org_number']) 

# Merge df_apartment and df_annualreport with an asof merge
# Assumption that the annual reports are posted at the end of December and people mostly see the annual reports from the year before 
df_apartment_with_annualreport = pd.merge_asof(
    df_apartment, df_annualreport,
    left_on='sell_year', right_on='fiscal_year_plus_one', # Match sell_year to fiscal_year_plus_one 
    by='org_number',                                      # Match on org_number before performing merge operation.
    direction='backward')                                 # Backward search selects the last row in df_annualreport whose 'on' key is less than or equal to the left's key.

# Merge df_apartment_with_annualreport with df_housingassociation on org_number
df = pd.merge(df_apartment_with_annualreport, df_housingassociation, on='org_number', how='left') 

# Dropping irrelevant columns,  sell_date, fiscal_year and fiscal_year_plus_one are only used for STEP 1 above
df = df.drop(['fiscal_year', 'sell_date', 'fiscal_year_plus_one'], axis=1) 

In [81]:
# Step 2 - Data Cleaning

# Creating unique IDs for missing values in org_number

# Define the starting number for new unique IDs
starting_number = 100000 

# Count the number of missing values in 'org_number', which is 69643 from data wrangler
missing_count = df['org_number'].isna().sum() 

# Generate new unique numbers starting from 'starting_number'
new_numbers = [f"{str(100000 + i).zfill(6)}-0000" for i in range(missing_count)] 

# Replace NaN values in 'org_number' with the new unique numbers
df.loc[df['org_number'].isna(), 'org_number'] = new_numbers 

In [82]:
# Step 2 - Data Cleaning Continued 

# Filling one missing value in municipality, postcode, primary_area and region, respectively based on address.

# Municipality
df['municipality'] = df['municipality'].fillna('Umeå')

# Postcode
df['postcode'] = df['postcode'].fillna(90736.0) 

# Primary_area 
df['primary_area'] = df['primary_area'].fillna('Ålidhem') 

# Region
df['region'] = df['region'].fillna('Västerbottens län')

In [83]:
# Step 2 - Data Cleaning Continued 

# Filling missing values in location 

# Sorting the remaining municipalities manually and assigning them to either Norra Sverige or Mellersta Sverige   
norra_sverige = ["Umeå"]
mellersta_sverige = ["Oxelösund", "Kungsör", "Hammarö", "Håbo"]

# Impute missing values in location
df['location'] = df.apply(
    lambda row: "Norra Sverige" if pd.isna(row['location']) and row['municipality'] in norra_sverige
    else ("Mellersta Sverige" if pd.isna(row['location']) and row['municipality'] in mellersta_sverige 
          else row['location']),
    axis=1)

In [84]:
# Step 2 - Data Cleaning Continued 

# Imputing missing values in housing_association_fee in order of proximity 
# Firstly, calculating the mean of housing_association_fee for the first grouping level
# Secondly, applying the mean to the rows that are in the specified group, and rounding housing_association_fee to the nearest whole number
# Continuing to the next grouping step and so on until all missing values are filled in

# Impute based on org_number first
mean_fee_by_org = df.groupby('org_number')['housing_association_fee'].transform('mean')
df['housing_association_fee'] = df['housing_association_fee'].fillna(mean_fee_by_org).round(0)

# Impute based on postcode secondly
mean_fee_by_postcode = df.groupby('postcode')['housing_association_fee'].transform('mean')
df['housing_association_fee'] = df['housing_association_fee'].fillna(mean_fee_by_postcode).round(0)

# Impute based on brokers description secondly
mean_fee_by_brokers_construction = df.groupby('brokers_description')['housing_association_fee'].transform('mean')
df['housing_association_fee'] = df['housing_association_fee'].fillna(mean_fee_by_brokers_construction).round(0)

# Impute based on municipality thirdly
mean_fee_by_municipality = df.groupby('municipality')['housing_association_fee'].transform('mean')
df['housing_association_fee'] = df['housing_association_fee'].fillna(mean_fee_by_municipality).round(0)

# Impute based on region fourthly
mean_fee_by_region = df.groupby('region')['housing_association_fee'].transform('mean')
df['housing_association_fee'] = df['housing_association_fee'].fillna(mean_fee_by_region).round(0)

# Remaining missing values are filled with the overall mean
overall_mean_fee = df['housing_association_fee'].mean()
df['housing_association_fee'] = df['housing_association_fee'].fillna(overall_mean_fee).round(0)

In [85]:
# Step 3 - Data Cleaning Continued 

# Imputing operating_cost in a similar way as the before, but now grouping by object type 

# Firstly, filling in missing values in operating_cost for non apartment objects only

# Fill by org_number
mean_operating_cost_by_org = df[df['object_type'] != 'Apartment'].groupby('org_number')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['org_number'].map(mean_operating_cost_by_org)).round(0)

# Fill by postcode
mean_operating_cost_by_postcode = df[df['object_type'] != 'Apartment'].groupby('postcode')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['postcode'].map(mean_operating_cost_by_postcode)).round(0)

# Fill by brokers_description
mean_operating_cost_by_brokers_description = df[df['object_type'] != 'Apartment'].groupby('brokers_description')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['brokers_description'].map(mean_operating_cost_by_brokers_description)).round(0)

# Fill by municipality
mean_operating_cost_by_municipality = df[df['object_type'] != 'Apartment'].groupby('municipality')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['municipality'].map(mean_operating_cost_by_municipality)).round(0)

# Fill by region
mean_operating_cost_by_region = df[df['object_type'] != 'Apartment'].groupby('region')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['region'].map(mean_operating_cost_by_region)).round(0)

# Secondly, filling in missing values in operating_cost for Apartments only

# Fill missing values in priority order 

# Fill by org_number
mean_operating_cost_by_org = df[df['object_type'] == 'Apartment'].groupby('org_number')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['org_number'].map(mean_operating_cost_by_org)).round(0)

# Fill by postcode
mean_operating_cost_by_postcode = df[df['object_type'] == 'Apartment'].groupby('postcode')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['postcode'].map(mean_operating_cost_by_postcode)).round(0)

# Fill by brokers_description
mean_operating_cost_by_brokers_description = df[df['object_type'] == 'Apartment'].groupby('brokers_description')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['brokers_description'].map(mean_operating_cost_by_brokers_description)).round(0)

# Fill by municipality
mean_operating_cost_by_municipality = df[df['object_type'] == 'Apartment'].groupby('municipality')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['municipality'].map(mean_operating_cost_by_municipality)).round(0)

# Fill by regions
mean_operating_cost_by_region = df[df['object_type'] == 'Apartment'].groupby('region')['operating_cost'].mean()
df['operating_cost'] = df['operating_cost'].fillna(df['region'].map(mean_operating_cost_by_region)).round(0)

In [86]:
# Step 2 - Data Cleaning Continued 

# Floor levels range from -6 up to 100.
# The negatives values seem to be wrongly specified, based on randomly looking up the sale information about some addresses.
# The highest floors are also wrongly specified, sinces the highest floor level in an apartment building in Sweden was 50 before 2023. 

# Adjusting floor level
df['floor'] = df['floor'].apply(lambda x: abs(x) if x <= -1 else (50 if x >= 50 else x)) 

# Calculate the mean floor value for apartments
mean_floor_apartment = df.loc[df['object_type'] == 'Apartment', 'floor'].mean()

# Impute missing floor values
df['floor'] = df.apply(
    lambda row: mean_floor_apartment 
    if pd.isna(row['floor']) and row['object_type'] == 'Apartment' 
    else (0 if pd.isna(row['floor']) else row['floor']),
    axis=1)

# Rounding floor to the nearest whole number
df['floor'] = df['floor'].round(0)

In [87]:
# Step 2 - Data Cleaning Continued 

# Imputing missing values amenities, key, has_fireplace and has_balcony

# Amenities 
# Check for missing values in 'amenities'
missing_mask = df['amenities'].isna() 

# One-hot encode the 'amenities' column
one_hot = df['amenities'].fillna('').str.get_dummies(sep=',') 

# Adding amenities_ infront the one-hot encoded variables 
one_hot = one_hot.add_prefix('amenities_') 

# Reintroduce missing values in the one-hot encoded columns
one_hot[missing_mask] = np.nan 

# Join the one-hot encoded columns with the original df
df = pd.concat([df, one_hot], axis=1) 

# Key 
# Check for missing values in 'key'
missing_mask = df['key'].isna() 

# One-hot encode the 'key' column
one_hot = df['key'].fillna('').str.get_dummies(sep=',') 

# Adding key_ infront the one-hot encoded variables 
one_hot = one_hot.add_prefix('key_') 

# Reintroduce missing values in the one-hot encoded columns
one_hot[missing_mask] = np.nan  

# Join the one-hot encoded columns with the original df
df = pd.concat([df, one_hot], axis=1) 

# Combining relevant columns created, from the step above, together 

# Filling in has_fireplace with values from key_fireplace and amenities_fireplace
df['has_fireplace'] = (df['has_fireplace']
                       .combine_first(df['key_fireplace'])         # Use key_fireplace if has_fireplace is NaN 
                       .combine_first(df['amenities_fireplace']))  # Use amenities_fireplace if both are NaN

# Dropping key_fireplace and amenities_fireplace
df = df.drop(['key_fireplace', 'amenities_fireplace'], axis=1) 

# Filling in has_balcony with values from amenities_balcony and key_balcony
df['has_balcony'] = (
    df['has_balcony']
    .combine_first(df['amenities_balcony'])  # Use amenities_balcony if has_balcony is NaN
    .combine_first(df['key_balcony']))       # Use key_balcony if both are NaN

# Dropping amenities_balcony and key_balcony
df = df.drop(['amenities_balcony', 'key_balcony'], axis=1) 

# Filling in has_patio with values from amenities_patio and key_patio
df['has_patio'] = (
    df['has_patio']
    .combine_first(df['amenities_patio'])  # Use amenities_patio if has_patio is NaN
    .combine_first(df['key_patio']))       # Use key_patio if both are NaN

# Dropping amenities_patio and key_patio
df = df.drop(['amenities_patio', 'key_patio'], axis=1) 

# Filling in amenities_elevator with values from key_elevator
df['amenities_elevator'] = (
    df['amenities_elevator']
    .combine_first(df['key_elevator']))  # Use key_elevator if amenities_elevator is NaN

# Dropping original amenities and key columns, and key_elevator
df = df.drop(['amenities', 'key', 'key_elevator'], axis=1) 

# Renaming columns for consistency reasons
df.rename(columns={'amenities_elevator': 'has_elevator', 'amenities_[]': 'has_other_amenities'}, inplace=True)

# Imputing has_balcony, has_fireplace, has_patio, has_solar_panels, has_elevator, has_other_amenities
# List of binary features to impute
binary_features = ['has_balcony', 'has_fireplace', 'has_patio', 'has_solar_panels', 'has_elevator', 'has_other_amenities'] 

# Creating a for loop that fills in the missing values for each column in binary_features. Imputing based on org_number
for feature in binary_features:
    org_numbers_with_feature = df.loc[df[feature] == 1, 'org_number'].unique() # Identify org_numbers where the feature is 1
    df.loc[df['org_number'].isin(org_numbers_with_feature), feature] = 1       # Set the feature to 1 for all rows with those org_numbers

# Update 'has_balcony' to 0 where 'floor' is 0 and 'has_balcony' is not already NaN
df.loc[(df['floor'] == 0) & (df['has_balcony'].notna()), 'has_balcony'] = 0 

# Preparing for one hot encoding 

# Specififying which columns to transform into text string representation 
columns_to_transform = ['has_fireplace', 'has_balcony', 'has_patio', 'has_solar_panels', 'has_elevator', 'has_other_amenities']

# Creating a foor loop where 1 is changed to yes, 0 to no, and missing values are filled with unknown
for column in columns_to_transform:
    df[column] = df[column].replace({1: 'yes', 0: 'no'}).fillna('unknown')

In [88]:
# Step 2 - Data Cleaning Continued 

# Firstly, filling in missing values in living area for all non apartment objects only

# Firstly, filling in missing values for each object type, excluding Aparjectment, by object_type grouping and taking the mean for each object_type
mean_living_area = df[df['object_type'] != 'Apartment'].groupby('object_type')['living_area'].mean()

# Fill N/A values in 'living_area' based on the mean for specific object types, still excluding Apartment
df['living_area'] = df.apply(
    lambda row: mean_living_area[row['object_type']] 
    if pd.isna(row['living_area']) and row['object_type'] in mean_living_area 
    else row['living_area'],
    axis=1)

# Secondly, filling in missing values in living area for all Apartments only

# Define grouping columns for imputing living_area
group_columns = ['org_number', 'postcode', 'brokers_description', 'municipality']

# For loop that will iterate over the grouping columns, and that will calculate the mean living_area for Apartment for each group 
# and then impute missing values using the calculated means. 
for group_col in group_columns:
    
    # Calculate the mean living_area for Apartment for each group 
    mean_living_area = (
        df[df['object_type'] == 'Apartment']
        .groupby(group_col)['living_area']
        .mean())
    
    # Impute missing 'living_area' using the calculated means
    df['living_area'] = df.apply(
        lambda row: mean_living_area[row[group_col]]
        if pd.isna(row['living_area']) and row['object_type'] == 'Apartment' and row[group_col] in mean_living_area
        else row['living_area'],
        axis=1)
    
# Rounding asking_price to the nearest whole number
df['living_area'] = df['living_area'].round(0)

In [ ]:
# Step 2 - Data Cleaning Continued 

# Based on living_area that has been imputed, we will fill in missing values in asking price 

# Price per sqm which is based on imputed living_area
df['price_per_sqm'] = df['asking_price'] / df['living_area']

# Defining a list of columns in the order we want to iterate them in
group_columns = ['org_number', 'postcode', 'brokers_description' ,'municipality', 'region']

# Impute missing asking_price iteratively for each group
for group_col in group_columns:

    # Calculate the mean price_per_sqm for each group
    mean_price_per_sqm = df.groupby(group_col)['price_per_sqm'].transform(lambda x: x.mean() if x.notna().any() else np.nan)
    
    # Filling missing asking_price using mean price_per_sqm and living_area:
    df['asking_price'] = df['asking_price'].fillna(df['living_area'] * mean_price_per_sqm)

# Rounding asking_price to the nearest whole number
df['asking_price'] = df['asking_price'].round(0)

In [90]:
# Step 2 - Data Cleaning Continued 

# Convert 'sell_year' from integer to string
df['sell_year'] = df['sell_year'].astype(str) 

# Convert 'construction_year' from integer to string
df['construction_year'] = df['construction_year'].astype(str) 

# Fill missing values in association_tax_liability with unknown
df['association_tax_liability'] = df['association_tax_liability'].fillna('unknown')

# Dropping variables that were not imputed, because they either had to many missing values or a low correlation with sell_price
# And dropping unnecessary identification IDs too 
df = df.drop(['additional_area', 'agency_id', 'brokers_description', 'cover_photo_description','customer_area_description', 'district', 'energy_class', 
              'height', 'org_number', 'id2', 'index', 'is_new_construction', 'locality', 'plot_area', 'populated_area', 'street_address', 'width', 
              'price_per_sqm', 'municipality', 'number_of_units', 'total_living_area', 'total_plot_area', 'living_area'], axis=1)

In [91]:
# STEP 3 - Modelling 

# Sorting our df in descending order and putting NaNs at then end
df = df.sort_values(by=['sell_price'], ascending=False, na_position='first') 

# Extracting 'id' that will be used for the JSON fil
missing_sell_price_count = df['sell_price'].isna().sum()
df_id_predicted_sell_price = df['id'].head(missing_sell_price_count)

# Dropping 'id' from the df that will be used for modelling  
df = df.drop(['id'], axis=1)

# Automatically detect continuous variables except for 'sell_price' 
continuous_vars = [
    col for col in df.select_dtypes(include=['float64', 'int64']).columns
    if col != 'sell_price'
    ]

# Apply StandardScaler to the continuous variables detected 
df[continuous_vars] = StandardScaler().fit_transform(df[continuous_vars])

# One Hot Encoding
df = pd.get_dummies(df, dtype=int)

# Create a new df with rows where 'sell_price' is missing
df_with_null = df[df['sell_price'].isna()]

# Remove missing rows in sell_price from the original df
df_without_null = df[df['sell_price'].notna()]

In [92]:
# STEP 3 - Modelling 

# Outlier handling in sell_price. The dataset is filtered to include only those instances where the sale price falls between the 1st and 99th percentiles.
max_threshold = df_without_null['sell_price'].quantile(0.99)
min_threshold = df_without_null['sell_price'].quantile(0.01)
df_without_null = df_without_null[(df_without_null['sell_price'] < max_threshold) & (df_without_null['sell_price'] > min_threshold)]

# Features 
X = df_without_null.drop('sell_price', axis=1)  

# Target
y = df_without_null['sell_price']              

# Splitting the data into training (90%) and testing (10%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

In [93]:
# STEP 3 - Modelling 

# XGBoost

# Use XGBRegressor for regression
xgboost_model = xgb.XGBRegressor(
    subsample = 1.0, 
    n_estimators = 1000, 
    max_depth = 9, 
    learning_rate = 0.06, 
    colsample_bytree = 1.0, 
    random_state = 10)  

# Fitting the model to the training data
xgboost_model.fit(X_train, y_train)  

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=10, ...)

In [94]:
# STEP 4 - Prediction

# Creating a new dataframe from the df with missing values for the sell_price
X_with_null = df_with_null.drop('sell_price', axis=1)

# Predicting the missing sell_price values 
predicted_sell_price = xgboost_model.predict(X_with_null)

In [ ]:
# STEP 5 - JSON

# predicted_sell_price should have the same index as X_with_null
predicted_sell_price = pd.DataFrame(predicted_sell_price, index=X_with_null.index)

# Add the predictions back to the dataframe
X_with_null['predicted_sell_price'] = predicted_sell_price

# Combine with ID
final = pd.concat([df_id_predicted_sell_price, X_with_null], axis=1)

# Dropping everything except 'id' and 'predicted_sell_price'
final = final[['id', 'predicted_sell_price']]

# Here we convert the cleaned dataframe to a list of dictionaries:
list_of_dictionaries = final.to_dict(orient='records')

# Specifying the output file name to my student number as in the instructions:
output_filename = 'assignment3_25199.json'

# Writing the list of dictionaries to a JSON file with my assigned file name:
with open(output_filename, 'w') as file:
    json.dump(list_of_dictionaries, file, indent=2)

                                      id  predicted_sell_price
596198  3fccf28bb678790528321f4589748a65           1952873.375
596202  7a72ef2f0bdaa0fbb5d1dfd8332b732b           7987848.000
596218  a938919c37d288a62e1c3f03f2158728           2751635.250
596222  6cfec1ad18f17bf99da6514b6f300c4d           8261725.500
596223  41688fb0a6d94d4dc84d0d3164d34fad           3099260.000
...                                  ...                   ...
668790  ffbee97ea37e1e3fb66cfa6d12840910           2175783.250
668793  ffd1af5bf80631fc8d3fb34cc90e007d            582606.750
668794  ffdba22478f015bff377f2677748d683           2555724.500
668796  ffe14bd66d76d39954e2e85930f0f025           7730961.000
668798  ffedd6c01b74454dff0dee1375b1846e           3220350.250

[19255 rows x 2 columns]
